# Libraries

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action="ignore")
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Pre-processing

In [2]:
#!python ../Preprocessing.py
claims = pd.read_csv("./claims.csv", low_memory=False)

In [3]:
claims

,Unnamed: 0,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,IsOutpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,ClaimDuration,NoPhy,AllPhy,SameAttOper,AdmisDuration,AgeAtClm,TotalRev,ClmYear,ClmMonth,ClmWeek,InsCovRatio,RevPerDay,Chronic_Sum,code_all_nan
0,0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1943-01-01,NaN,1,1,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,1,6,False,False,False,6.0,66,27068.0,2009,4,15,0.960544,3866.857143,7,False
1,1,BENE16973,CLM565430,2009-09-06,2009-09-06,PRV55912,50,PHY365867,PHY327147,NaN,NaN,NaN,0.0,NaN,NaN,V7183,53081,78959,4280,E8788,78079,79902,25002,71848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1932-05-01,NaN,1,1,0,39,310,12,12,1,1,1,0,0,0,0,1,0,0,0,24000,2136,450,200,1,0,False,True,False,NaN,77,50.0,2009,9,36,1.000000,50.000000,4,False
2,2,BENE17521,CLM34721,2009-01-20,2009-02-01,PRV55912,19000,PHY349293,PHY370861,PHY363291,2009-01-20,45340,1068.0,2009-02-01,987,4240,2639,2948,40390,45821,28489,5854,2753,E9305,NaN,7769.0,5849.0,NaN,NaN,NaN,NaN,0,1913-12-01,NaN,0,1,0,39,230,12,12,1,1,1,0,1,0,0,1,0,0,0,19000,1068,100,20,1,12,False,True,False,12.0,95,20068.0,2009,1,4,0.946781,1543.692308,5,False
3,3,BENE21718,CLM72336,2009-10-17,2009-11-04,PRV55912,17000,PHY334706,PHY334706,NaN,2009-10-17,V5789,1068.0,2009-11-04,941,V5789,4168,73313,7812,7993,78830,72273,43812,4019,NaN,9338.0,NaN,NaN,NaN,NaN,NaN,0,1922-10-01,NaN,1,1,0,39,600,12,12,0,0,0,0,0,0,0,1,1,0,0,17000,1068,1050,540,1,18,False,True,True,18.0,87,18068.0,2009,10,42,0.940890,950.947368,2,False
4,4,BENE22934,CLM73394,2009-10-25,2009-10-29,PRV55912,13000,PHY390614,PHY323689,PHY363291,2009-10-25,71946,1068.0,2009-10-29,506,71535,71960,4019,V1202,4240,2449,2768,NaN,NaN,NaN,8154.0,NaN,NaN,NaN,NaN,NaN,0,1930-07-01,NaN,0,1,0,39,280,12,12,0,1,0,0,1,0,1,1,1,1,0,27000,2136,450,160,1,4,False,True,False,4.0,79,14068.0,2009,10,43,0.924083,2813.600000,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,558206,BENE154147,CLM394122,2009-06-02,2009-06-04,PRV54050,500,PHY317497,NaN,PHY317497,NaN,NaN,0.0,NaN,NaN,71698,71941,7862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1925-01-01,NaN,0,1,0,23,400,12,12,0,0,0,0,0,0,0,1,1,0,0,0,0,890,120,0,2,False,False,False,NaN,84,500.0,2009,6,23,1.000000,166.666667,2,False
558207,558207,BENE154687,CLM184358,2009-02-08,2009-02-08,PRV54302,3300,PHY376238,PHY376238,NaN,NaN,99639,0.0,NaN,NaN,99661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1927-05-01,NaN,1,2,0,25,150,12,12,1,0,1,0,0,0,1,1,0,1,0,0,0,4400,220,0,0,False,True,True,NaN,82,3300.0,2009,2,6,1.000000,3300.000000,5,False
558208,558208,BENE157378,CLM460770,2009-07-09,2009-07-29,PRV51577,2100,PHY338096,NaN,NaN,NaN,NaN,0.0,NaN,NaN,5854,7907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Feature Engineering

- Ratio of inpatients that receive both in/out patient service

In [4]:
uniqueInpatientBeneID = claims[claims['IsOutpatient']==0]['BeneID'].unique().tolist()
uniqueOutpatientBeneID = claims[claims['IsOutpatient']==1]['BeneID'].unique().tolist()

uniqueInOutpatientBeneID = set(uniqueInpatientBeneID).intersection(set(uniqueOutpatientBeneID))

In [5]:
condlist = [(claims['BeneID'].isin(uniqueInOutpatientBeneID))]
choicelist = [1]
claims['isInOutBeneID'] = np.select(condlist, choicelist, default=0)

claims['isInOutBeneID'].value_counts()

0    397329
1    160882
Name: isInOutBeneID, dtype: int64

In [6]:
# totalInpatientClaimCount_Provider = claims[claims['IsOutpatient']==0].groupby('Provider', as_index=False
#                                          )['ClaimID'].count().rename(columns={'ClaimID':'TotalInpatientClaimCount'})


# inpatientInOutClaimCount_Provider = claims[(claims['isInOutBeneID']==1) & (claims['IsOutpatient']==0)].groupby('Provider', as_index=False
#                                           )['ClaimID'].count().rename(columns={'ClaimID':'TotalInOutpatientClaimCount'})


# ratioInpatientClaims_InOutservice = totalInpatientClaimCount_Provider.join(inpatientInOutClaimCount_Provider.set_index('Provider'), on='Provider')
# ratioInpatientClaims_InOutservice['inpatientsRatioClaims_InOutservice'] = ratioInpatientClaims_InOutservice['TotalInOutpatientClaimCount'] / ratioInpatientClaims_InOutservice['TotalInpatientClaimCount']

# ratioInpatientClaims_InOutservice

- Ratio of outpatients that receive both in/out patient service

In [7]:
totalOutpatientClaimCount_Provider = claims[claims['IsOutpatient']==1].groupby('Provider', as_index=False
                                         )['ClaimID'].count().rename(columns={'ClaimID':'TotalOutpatientClaimCount'})


outpatientInOutClaimCount_Provider = claims[(claims['isInOutBeneID']==1) & (claims['IsOutpatient']==1)].groupby('Provider', as_index=False
                                          )['ClaimID'].count().rename(columns={'ClaimID':'TotalInOutpatientClaimCount'})


ratioOutpatientClaims_InOutservice = totalOutpatientClaimCount_Provider.join(outpatientInOutClaimCount_Provider.set_index('Provider'), on='Provider')
ratioOutpatientClaims_InOutservice['outpatientsRatioClaims_InOutservice'] = ratioOutpatientClaims_InOutservice['TotalInOutpatientClaimCount'] / ratioOutpatientClaims_InOutservice['TotalOutpatientClaimCount']

ratioOutpatientClaims_InOutservice



,Provider,TotalOutpatientClaimCount,TotalInOutpatientClaimCount,outpatientsRatioClaims_InOutservice
0,PRV51001,20,6.0,0.300000
1,PRV51003,70,12.0,0.171429
2,PRV51004,149,31.0,0.208054
3,PRV51005,1165,289.0,0.248069
4,PRV51007,69,18.0,0.260870
...,...,...,...,...
5007,PRV57759,28,6.0,0.214286
5008,PRV57760,22,8.0,0.363636
5009,PRV57761,82,28.0,0.341463
5010,PRV57762,1,1.0,1.000000


- Percentage of patients that receive both in/out patient service

In [8]:
claims['percentage_InOutpatients'] = claims['ClmAdmitDiagnosisCode'].isin(top5admtcode)
percentage_InOutpatients = claims.groupby('Provider')[['percentage_InOutpatients']].mean().reset_index()
percentage_InOutpatients

NameError: name 'top5admtcode' is not defined

- Ratio of claims that didn't have any physician involved

In [ ]:
condlist = [(claims['AttendingPhysician'].notna()|
             claims['OperatingPhysician'].notna()|
             claims['OperatingPhysician'].notna())]
choicelist = ['1']
claims['PhysicianExists'] = np.select(condlist, choicelist, default='0')

totalClaimCount_Provider = claims.groupby('Provider', as_index=False
                                         )['ClaimID'].count().rename(columns={'ClaimID':'TotalClaimCount'})
noPhysicianClaimCount_Provider = claims[claims['PhysicianExists'] == '0'].groupby(
    'Provider', as_index=False)['ClaimID'].count().rename(columns={'ClaimID':'noPhysicianClaimCount'})

ratioClaims_NoPhysician = totalClaimCount_Provider.join(noPhysicianClaimCount_Provider.set_index('Provider'), on='Provider')
ratioClaims_NoPhysician['percentage_noPhysician'] = ratioClaims_NoPhysician['noPhysicianClaimCount'] / ratioClaims_NoPhysician['TotalClaimCount']

percentage_noPhysician = ratioClaims_NoPhysician[['Provider','percentage_noPhysician']]
percentage_noPhysician


- Ratio of inpatient claims with top 5 admtcode (from PotentialFraud)

In [ ]:
# top 5 admtcode
claims['ClmAdmitDiagnosisCode'].value_counts().sort_values(ascending=False).head(5)
top5admtcode = ['V7612','42731','78605','4019','25000']

claims['isTop5admtcode'] = claims['ClmAdmitDiagnosisCode'].isin(top5admtcode)

claims[claims['IsOutpatient']==0].groupby('Provider')[['isTop5admtcode']].mean().reset_index()


- Ratio of outpatient claims with top 5 admtcode (from PotentialFraud)

In [ ]:
claims[claims['IsOutpatient']==1].groupby('Provider')[['isTop5admtcode']].mean().reset_index()

- Percentage top 5 admtcode

In [ ]:
percentage_top5admtcode = claims.groupby('Provider')[['isTop5admtcode']].mean().reset_index()
percentage_top5admtcode

- Ratio of patients going to different hospitals (75% threshold)

In [ ]:
claims['isTop5admtcode'] = claims['ClmAdmitDiagnosisCode'].isin(top5admtcode)
percentage_patientsDiffHospitals = claims.groupby('Provider')[['isTop5admtcode']].mean().reset_index()
percentage_patientsDiffHospitals

---------

# Make 'Provider' csv

In [ ]:
claims.Provider.value_counts()

In [ ]:
# percentage_InOutpatients
# percentage_noPhysician
# percentage_top5admtcode

providers = percentage_InOutpatients.join(percentage_noPhysician.set_index('Provider'), on='Provider'
                             ).join(percentage_top5admtcode.set_index('Provider'), on='Provider')

providers

In [ ]:
providers.fillna(0)

In [ ]:
providers.to_csv(r'./data/providers.csv', index = False)